In [126]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/Covid Data.csv')

In [127]:
# Prikaz prvih nekoliko redova 
print(df.head())

# Broj redova pre čišćenja
print(f"Početni broj redova: {df.shape[0]}")


   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2             1    1             1  03/05/2020       97          1   
1      2             1    2             1  03/06/2020       97          1   
2      2             1    2             2  09/06/2020        1          2   
3      2             1    1             1  12/06/2020       97          2   
4      2             1    2             1  21/06/2020       97          2   

   AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HIPERTENSION  OTHER_DISEASE  \
0   65         2         2  ...       2        2             1              2   
1   72        97         2  ...       2        2             1              2   
2   55        97         1  ...       2        2             2              2   
3   53         2         2  ...       2        2             2              2   
4   68        97         1  ...       2        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO

In [128]:
# PROVERA NEDOSTAJUĆIH/NEPOZNATIH VREDNOSTI
# Iz dokumentacije dataseta vidimo da 97, 98 i 99 predstavljaju nedostajuće vrednosti
# Ove vrednosti tretiramo kao NaN da bismo videli pravo stanje
# Vodimo računa da za kolonu 'AGE' mogu postojati pacijenti sa ovim vrednostima godina, pa cemo nju zasebno proveriti!
# Zasebno ćemo proveriti i DATE_DIED zbog specifičnog formata

print(f"Kolona 'AGE' ima {df['AGE'].isna().sum()} nedostajućih vrednosti")
print(f"Kolona 'DATE_DIED' ima {df['DATE_DIED'].isnull().sum()} nedostajućih vrednosti")

cols_to_check = [col for col in df.columns if col not in ['AGE', 'DATE_DIED']]

missing_report = pd.DataFrame(index=cols_to_check)

missing_report['Broj_vrednosti_97'] = (df[cols_to_check] == 97).sum()
missing_report['Broj_vrednosti_98'] = (df[cols_to_check] == 98).sum()
missing_report['Broj_vrednosti_99'] = (df[cols_to_check] == 99).sum()
missing_report['Ukupno_nedostajućih'] = missing_report['Broj_vrednosti_97'] + missing_report['Broj_vrednosti_99'] + missing_report['Broj_vrednosti_98']
missing_report['Procenat_%'] = (missing_report['Ukupno_nedostajućih'] / len(df)) * 100

missing_report = missing_report.sort_values(by='Procenat_%', ascending=False)

print("---IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA ---")
print(missing_report)

# Iz izvestaja vidimo 
# INTUBED i ICU -> Ogroman procenat nedostajućih (preko 80%) i to je zato što je ovde vrednost 97 za sve pacijente koji nisu hospitalizovani (što je većina)
# PREGNANCY -> Oko 50% "nedostajućih" vrednosti jer se podatak ne odnosi na muškarce
# PNEUMONIA -> Ovde imamo oko 1% nedostajućih vrednosti
# OSTALO -> Zanemarljivo malo nedostajućih vrednosti

Kolona 'AGE' ima 0 nedostajućih vrednosti
Kolona 'DATE_DIED' ima 0 nedostajućih vrednosti
---IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA ---
                      Broj_vrednosti_97  Broj_vrednosti_98  Broj_vrednosti_99  \
ICU                              848544                  0               7488   
INTUBED                          848544                  0               7325   
PREGNANT                         523511               3754                  0   
PNEUMONIA                             0                  0              16003   
OTHER_DISEASE                         0               5045                  0   
INMSUPR                               0               3404                  0   
DIABETES                              0               3338                  0   
TOBACCO                               0               3220                  0   
HIPERTENSION                          0               3104                  0   
CARDIOVASCULAR                        0               307

In [129]:
#PROVERA DUPLIKATA 

duplicate_count = df.duplicated().sum()
print(f"Broj istih redova u datasetu: {duplicate_count}")

# Iako vidimo veliki broj identičnih zapisa, oni se ne možemo da smatramo duplikatima jer dataset ne sadrži jedinstveni identifikator
# pacijenta na osnovu kog bismo pouzdano mogli da znamo da se radi o dupliranom unosu istog pacijenta. 
# Identitet zapisa ne može se pouzdano utvrditi, pa ćemo pretpostavljati da zapisi predstavljaju različite pojedince sa istim karakteristikama.

Broj istih redova u datasetu: 812049


In [144]:
# PROVERA NEKONZISTENTNOSTI PODATAKA

# provera godina starosti
print("-----PROVERA STAROSTI-----")
invalid_age = df[(df['AGE'] < 0)] #ne uključujemo i 0 jer su to bebe i iskrivilo bi starosnu distribuciju
print(f"Broj zapisa sa loše unetim godinama: {len(invalid_age)}")

#PROVERA KATEGORIJSKIH PODATAKA U KOJIMA NEMAMO NEDOSTAJUCIH VREDNOSTI
print(f"\n-----PROVERA KATEGORIJSKIH NENEDOSTAJUĆIH PODATAKA-----\n")
# provera pola - dozvoljene su samo vrednosti 1 i 2
print("-----PROVERA POLA-----")
print(df['SEX'].value_counts())

print("-----PROVERA KATEGORIJA ZA KUĆU I BOLNICU-----")
print(df['PATIENT_TYPE'].value_counts())

print(f"-----------------------------------------\n")


# provera da li su pacijenti koji nisu hospitalizovani bili na intenzivnoj nezi
print(f"\n-----PROVERA HOSPITALIZACIJE I INTENZIVNE NEGE-----")
invalid_icu = df[(df['PATIENT_TYPE'] == 1) & (df['ICU'] == 1)]
invalid_intubed = df[(df['PATIENT_TYPE'] == 1) & (df['INTUBED'] == 1)]

print(f"Neispravni ICU zapisi: {len(invalid_icu)}")
print(f"Neispravni INTUBED zapisi: {len(invalid_intubed)}")

# provera da li imamo trudnih muškaraca
print(f"\n-----PROVERA TRUDNOĆE -----")
invalid_pregnancy_men = df[(df['SEX'] == 2) & (df['PREGNANT'] == 1)]
print(f"Neispravni zapisi o trudnoći muškaraca: {len(invalid_pregnancy_men)}")

invalid_pregnancy_baby = df[(df['AGE'] <= 5) & (df['PREGNANT'] == 1)]
print(f"Neispravni zapisi o trudnoći male dece i beba: {len(invalid_pregnancy_baby)}")

# provera da li imamo beba pušača
print(f"\n-----PROVERA PUŠAČA-----")
invalid_smokers = df[(df['AGE'] == 0) & (df['TOBACCO'] == 1)]
print(f"Neispravni zapisi o pušačima (bebe pušači): {len(invalid_smokers)}")






-----PROVERA STAROSTI-----
Broj zapisa sa loše unetim godinama: 0

-----PROVERA KATEGORIJSKIH NENEDOSTAJUĆIH PODATAKA-----

-----PROVERA POLA-----
SEX
1    525064
2    523511
Name: count, dtype: int64
-----PROVERA KATEGORIJA ZA KUĆU I BOLNICU-----
PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64
-----------------------------------------


-----PROVERA HOSPITALIZACIJE I INTENZIVNE NEGE-----
Neispravni ICU zapisi: 0
Neispravni INTUBED zapisi: 0

-----PROVERA TRUDNOĆE -----
Neispravni zapisi o trudnoći muškaraca: 0
Neispravni zapisi o trudnoći male dece i beba: 8

-----PROVERA PUŠAČA-----
Neispravni zapisi o pušačima (bebe pušači): 0


In [145]:
# KOREKCIJE 
# -> bebe ne mogu biti pušači
df.loc[(df['AGE'] == 0) & (df['TOBACCO'] == 1), 'TOBACCO'] = 0

# -> deca ispod 5 godina ne mogu biti trudna
df.loc[(df['AGE'] <= 5) & (df['PREGNANT'] == 1), 'PREGNANT'] = 2